In [41]:
import sys

import gym
import numpy as np

from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import A2C, DQN

# using wredsen's symbtools fork (https://github.com/wredsen/symbtools @ DAE_statefeedback), assuming repos SA-Wrede and symbtools share the same parent directory
sys.path.append('../../symbtools/')
import symbtools as st
import pickle

In [52]:
# load model etc. from pickle of flatness analysis notebook
with open("../flatness_notebooks/single_crane_model.pcl", "rb") as pfile:
    data = pickle.load(pfile)
    locals().update(data)

In [53]:
mod.eqns

Matrix([
[       m2*pddot1 - tau2*(p1 - q1)/sqrt(p2**2 + (p1 - q1)**2)],
[       g*m2 + m2*pddot2 - p2*tau2/sqrt(p2**2 + (p1 - q1)**2)],
[m1*qddot1 - tau1 + tau2*(p1 - q1)/sqrt(p2**2 + (p1 - q1)**2)]])

## Utility functions for Gym environment

In [2]:
"""
Utility functions used for classic control environments.
"""

from typing import Optional, SupportsFloat, Tuple


def verify_number_and_cast(x: SupportsFloat) -> float:
    """Verify parameter is a single number and cast to a float."""
    try:
        x = float(x)
    except (ValueError, TypeError):
        raise ValueError(f"An option ({x}) could not be converted to a float.")
    return x


def maybe_parse_reset_bounds(
    options: Optional[dict], default_low: float, default_high: float
) -> Tuple[float, float]:
    """
    This function can be called during a reset() to customize the sampling
    ranges for setting the initial state distributions.

    Args:
      options: Options passed in to reset().
      default_low: Default lower limit to use, if none specified in options.
      default_high: Default upper limit to use, if none specified in options.

    Returns:
      Tuple of the lower and upper limits.
    """
    if options is None:
        return default_low, default_high

    low = options.get("low") if "low" in options else default_low
    high = options.get("high") if "high" in options else default_high

    # We expect only numerical inputs.
    low = verify_number_and_cast(low)
    high = verify_number_and_cast(high)
    if low > high:
        raise ValueError(
            f"Lower bound ({low}) must be lower than higher bound ({high})."
        )

    return low, high

In [3]:
"""A utility class to collect render frames from a function that computes a single frame."""
from typing import Any, Callable, List, Optional, Set

# list of modes with which render function returns None
NO_RETURNS_RENDER = {"human"}

# list of modes with which render returns just a single frame of the current state
SINGLE_RENDER = {"single_rgb_array", "single_depth_array", "single_state_pixels"}


class Renderer:
    """This class serves to easily integrate collection of renders for environments that can computes a single render.

    To use this function:
    - instantiate this class with the mode and the function that computes a single frame
    - call render_step method each time the frame should be saved in the list
      (usually at the end of the step and reset methods)
    - call get_renders whenever you want to retrieve renders
      (usually in the render method)
    - call reset to clean the render list
      (usually in the reset method of the environment)
    """

    def __init__(
        self,
        mode: Optional[str],
        render: Callable[[str], Any],
        no_returns_render: Optional[Set[str]] = None,
        single_render: Optional[Set[str]] = None,
    ):
        """Instantiates a Renderer object.

        Args:
            mode (Optional[str]): Way to render
            render (Callable[[str], Any]): Function that receives the mode and computes a single frame
            no_returns_render (Optional[Set[str]]): Set of render modes that don't return any value.
                The default value is the set {"human"}.
            single_render (Optional[Set[str]]): Set of render modes that should return a single frame.
                The default value is the set {"single_rgb_array", "single_depth_array", "single_state_pixels"}.
        """
        if no_returns_render is None:
            no_returns_render = NO_RETURNS_RENDER
        if single_render is None:
            single_render = SINGLE_RENDER

        self.no_returns_render = no_returns_render
        self.single_render = single_render
        self.mode = mode
        self.render = render
        self.render_list = []

    def render_step(self) -> None:
        """Computes a frame and save it to the render collection list.

        This method should be usually called inside environment's step and reset method.
        """
        if self.mode is not None and self.mode not in self.single_render:
            render_return = self.render(self.mode)
            if self.mode not in self.no_returns_render:
                self.render_list.append(render_return)

    def get_renders(self) -> Optional[List]:
        """Pops all the frames from the render collection list.

        This method should be usually called in the environment's render method to retrieve the frames collected till this time step.
        """
        if self.mode in self.single_render:
            return self.render(self.mode)
        elif self.mode is not None and self.mode not in self.no_returns_render:
            renders = self.render_list
            self.render_list = []
            return renders

    def reset(self):
        """Resets the render collection list.

        This method should be usually called inside environment's reset method.
        """
        self.render_list = []

## Gym environment for crane

In [4]:
"""
Classic cart-pole system implemented by Rich Sutton et al.
Copied from http://incompleteideas.net/sutton/book/code/pole.c
permalink: https://perma.cc/C9ZM-652R
"""
import math
from typing import Optional, Union

import numpy as np

import gym
from gym import logger, spaces
#from gym.envs.classic_control import utils
from gym.error import DependencyNotInstalled
#from gym.utils.renderer import Renderer


class CartPoleEnv(gym.Env):
    """
    ### Description

    This environment corresponds to the version of the cart-pole problem described by Barto, Sutton, and Anderson in
    ["Neuronlike Adaptive Elements That Can Solve Difficult Learning Control Problem"](https://ieeexplore.ieee.org/document/6313077).
    A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track.
    The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces
     in the left and right direction on the cart.

    ### Action Space

    The action is a `ndarray` with shape `(1,)` which can take values `{0, 1}` indicating the direction
     of the fixed force the cart is pushed with.

    | Num | Action                 |
    |-----|------------------------|
    | 0   | Push cart to the left  |
    | 1   | Push cart to the right |

    **Note**: The velocity that is reduced or increased by the applied force is not fixed and it depends on the angle
     the pole is pointing. The center of gravity of the pole varies the amount of energy needed to move the cart underneath it

    ### Observation Space

    The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:

    | Num | Observation           | Min                 | Max               |
    |-----|-----------------------|---------------------|-------------------|
    | 0   | Cart Position         | -4.8                | 4.8               |
    | 1   | Cart Velocity         | -Inf                | Inf               |
    | 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
    | 3   | Pole Angular Velocity | -Inf                | Inf               |

    **Note:** While the ranges above denote the possible values for observation space of each element,
        it is not reflective of the allowed values of the state space in an unterminated episode. Particularly:
    -  The cart x-position (index 0) can be take values between `(-4.8, 4.8)`, but the episode terminates
       if the cart leaves the `(-2.4, 2.4)` range.
    -  The pole angle can be observed between  `(-.418, .418)` radians (or **±24°**), but the episode terminates
       if the pole angle is not in the range `(-.2095, .2095)` (or **±12°**)

    ### Rewards

    Since the goal is to keep the pole upright for as long as possible, a reward of `+1` for every step taken,
    including the termination step, is allotted. The threshold for rewards is 475 for v1.

    ### Starting State

    All observations are assigned a uniformly random value in `(-0.05, 0.05)`

    ### Episode End

    The episode ends if any one of the following occurs:

    1. Termination: Pole Angle is greater than ±12°
    2. Termination: Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)
    3. Truncation: Episode length is greater than 500 (200 for v0)

    ### Arguments

    ```
    gym.make('CartPole-v1')
    ```

    No additional arguments are currently supported.
    """

    metadata = {
        "render_modes": ["human", "rgb_array", "single_rgb_array"],
        "render_fps": 50,
    }

    def __init__(self, render_mode: Optional[str] = None):
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = self.masspole + self.masscart
        self.length = 0.5  # actually half the pole's length
        self.polemass_length = self.masspole * self.length
        self.force_mag = 10.0
        self.tau = 0.02  # seconds between state updates
        self.kinematics_integrator = "euler"

        # Angle at which to fail the episode
        self.theta_threshold_radians = 12 * 2 * math.pi / 360
        self.x_threshold = 2.4

        # Angle limit set to 2 * theta_threshold_radians so failing observation
        # is still within bounds.
        high = np.array(
            [
                self.x_threshold * 2,
                np.finfo(np.float32).max,
                self.theta_threshold_radians * 2,
                np.finfo(np.float32).max,
            ],
            dtype=np.float32,
        )

        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Box(-high, high, dtype=np.float32)

        self.render_mode = render_mode
        self.renderer = Renderer(self.render_mode, self._render)

        self.screen_width = 600
        self.screen_height = 400
        self.screen = None
        self.clock = None
        self.isopen = True
        self.state = None

        self.steps_beyond_terminated = None

    def step(self, action):
        err_msg = f"{action!r} ({type(action)}) invalid"
        assert self.action_space.contains(action), err_msg
        assert self.state is not None, "Call reset before using step method."
        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag if action == 1 else -self.force_mag
        costheta = math.cos(theta)
        sintheta = math.sin(theta)

        # For the interested reader:
        # https://coneural.org/florian/papers/05_cart_pole.pdf
        temp = (
            force + self.polemass_length * theta_dot**2 * sintheta
        ) / self.total_mass
        thetaacc = (self.gravity * sintheta - costheta * temp) / (
            self.length * (4.0 / 3.0 - self.masspole * costheta**2 / self.total_mass)
        )
        xacc = temp - self.polemass_length * thetaacc * costheta / self.total_mass

        if self.kinematics_integrator == "euler":
            x = x + self.tau * x_dot
            x_dot = x_dot + self.tau * xacc
            theta = theta + self.tau * theta_dot
            theta_dot = theta_dot + self.tau * thetaacc
        else:  # semi-implicit euler
            x_dot = x_dot + self.tau * xacc
            x = x + self.tau * x_dot
            theta_dot = theta_dot + self.tau * thetaacc
            theta = theta + self.tau * theta_dot

        self.state = (x, x_dot, theta, theta_dot)

        terminated = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            # Pole just fell!
            self.steps_beyond_terminated = 0
            reward = 1.0
        else:
            if self.steps_beyond_terminated == 0:
                logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0.0

        self.renderer.render_step()
        return np.array(self.state, dtype=np.float32), reward, terminated, {"info": False}#False, {}

    def reset(
        self,
        *,
        seed: Optional[int] = None,
        options: Optional[dict] = None,
    ):
        #super().reset()(seed=seed)
        # Note that if you use custom reset bounds, it may lead to out-of-bound
        # state/observations.
        low, high = maybe_parse_reset_bounds(
            options, -0.05, 0.05  # default low
        )  # default high
        self.state = np.random.uniform(low=low, high=high, size=(4,))
        self.steps_beyond_terminated = None
        self.renderer.reset()
        self.renderer.render_step()
        return np.array(self.state, dtype=np.float32)#, {}

    def render(self):
        return self.renderer.get_renders()

    def _render(self, mode="human"):
        assert mode in self.metadata["render_modes"]
        try:
            import pygame
            from pygame import gfxdraw
        except ImportError:
            raise DependencyNotInstalled(
                "pygame is not installed, run `pip install gym[classic_control]`"
            )

        if self.screen is None:
            pygame.init()
            if mode == "human":
                pygame.display.init()
                self.screen = pygame.display.set_mode(
                    (self.screen_width, self.screen_height)
                )
            else:  # mode in {"rgb_array", "single_rgb_array"}
                self.screen = pygame.Surface((self.screen_width, self.screen_height))
        if self.clock is None:
            self.clock = pygame.time.Clock()

        world_width = self.x_threshold * 2
        scale = self.screen_width / world_width
        polewidth = 10.0
        polelen = scale * (2 * self.length)
        cartwidth = 50.0
        cartheight = 30.0

        if self.state is None:
            return None

        x = self.state

        self.surf = pygame.Surface((self.screen_width, self.screen_height))
        self.surf.fill((255, 255, 255))

        l, r, t, b = -cartwidth / 2, cartwidth / 2, cartheight / 2, -cartheight / 2
        axleoffset = cartheight / 4.0
        cartx = x[0] * scale + self.screen_width / 2.0  # MIDDLE OF CART
        carty = 100  # TOP OF CART
        cart_coords = [(l, b), (l, t), (r, t), (r, b)]
        cart_coords = [(c[0] + cartx, c[1] + carty) for c in cart_coords]
        gfxdraw.aapolygon(self.surf, cart_coords, (0, 0, 0))
        gfxdraw.filled_polygon(self.surf, cart_coords, (0, 0, 0))

        l, r, t, b = (
            -polewidth / 2,
            polewidth / 2,
            polelen - polewidth / 2,
            -polewidth / 2,
        )

        pole_coords = []
        for coord in [(l, b), (l, t), (r, t), (r, b)]:
            coord = pygame.math.Vector2(coord).rotate_rad(-x[2])
            coord = (coord[0] + cartx, coord[1] + carty + axleoffset)
            pole_coords.append(coord)
        gfxdraw.aapolygon(self.surf, pole_coords, (202, 152, 101))
        gfxdraw.filled_polygon(self.surf, pole_coords, (202, 152, 101))

        gfxdraw.aacircle(
            self.surf,
            int(cartx),
            int(carty + axleoffset),
            int(polewidth / 2),
            (129, 132, 203),
        )
        gfxdraw.filled_circle(
            self.surf,
            int(cartx),
            int(carty + axleoffset),
            int(polewidth / 2),
            (129, 132, 203),
        )

        gfxdraw.hline(self.surf, 0, self.screen_width, carty, (0, 0, 0))

        self.surf = pygame.transform.flip(self.surf, False, True)
        self.screen.blit(self.surf, (0, 0))
        if mode == "human":
            pygame.event.pump()
            self.clock.tick(self.metadata["render_fps"])
            pygame.display.flip()

        elif mode in {"rgb_array", "single_rgb_array"}:
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
            )

    def close(self):
        if self.screen is not None:
            import pygame

            pygame.display.quit()
            pygame.quit()
            self.isopen = False

In [11]:
# environment without renderings for training
env = CartPoleEnv()
# environment with renderings for validating
env_rendering = CartPoleEnv(render_mode = "human")

In [12]:
env.reset()
env.step(env.action_space.sample())
env.observation_space
check_env(env)

array([-0.04043068,  0.01397029, -0.04246754,  0.01398392], dtype=float32)

## Learning the model

In [31]:
# Learning!
model = A2C('MlpPolicy', env, verbose=1, tensorboard_log="./a2c_cartpole_tensorboard/")
model.learn(total_timesteps=400000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./a2c_cartpole_tensorboard/A2C_3
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 26       |
|    ep_rew_mean        | 26       |
| time/                 |          |
|    fps                | 1504     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.679   |
|    explained_variance | 0.139    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -3.43    |
|    value_loss         | 28.9     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 27.5     |
|    ep_rew_mean        | 27.5     |
| time/                 |          |
|    fps                | 1537     |
|    iterat

-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 66.3      |
|    ep_rew_mean        | 66.3      |
| time/                 |           |
|    fps                | 1563      |
|    iterations         | 1400      |
|    time_elapsed       | 4         |
|    total_timesteps    | 7000      |
| train/                |           |
|    entropy_loss       | -0.406    |
|    explained_variance | -5.17e-05 |
|    learning_rate      | 0.0007    |
|    n_updates          | 1399      |
|    policy_loss        | 0.166     |
|    value_loss         | 1.18      |
-------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 70.2     |
|    ep_rew_mean        | 70.2     |
| time/                 |          |
|    fps                | 1565     |
|    iterations         | 1500     |
|    time_elapsed       | 4        |
|    total_timesteps    | 7500     |
| train/             

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 122      |
|    ep_rew_mean        | 122      |
| time/                 |          |
|    fps                | 1564     |
|    iterations         | 2700     |
|    time_elapsed       | 8        |
|    total_timesteps    | 13500    |
| train/                |          |
|    entropy_loss       | -0.306   |
|    explained_variance | -0.217   |
|    learning_rate      | 0.0007   |
|    n_updates          | 2699     |
|    policy_loss        | 2.82e-05 |
|    value_loss         | 1.09e-08 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 125      |
|    ep_rew_mean        | 125      |
| time/                 |          |
|    fps                | 1563     |
|    iterations         | 2800     |
|    time_elapsed       | 8        |
|    total_timesteps    | 14000    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 174      |
|    ep_rew_mean        | 174      |
| time/                 |          |
|    fps                | 1557     |
|    iterations         | 4000     |
|    time_elapsed       | 12       |
|    total_timesteps    | 20000    |
| train/                |          |
|    entropy_loss       | -0.332   |
|    explained_variance | -102     |
|    learning_rate      | 0.0007   |
|    n_updates          | 3999     |
|    policy_loss        | -5.21    |
|    value_loss         | 472      |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 176      |
|    ep_rew_mean        | 176      |
| time/                 |          |
|    fps                | 1559     |
|    iterations         | 4100     |
|    time_elapsed       | 13       |
|    total_timesteps    | 20500    |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 97.6     |
|    ep_rew_mean        | 97.6     |
| time/                 |          |
|    fps                | 1571     |
|    iterations         | 5400     |
|    time_elapsed       | 17       |
|    total_timesteps    | 27000    |
| train/                |          |
|    entropy_loss       | -0.312   |
|    explained_variance | -23.9    |
|    learning_rate      | 0.0007   |
|    n_updates          | 5399     |
|    policy_loss        | -0.112   |
|    value_loss         | 1.18     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 86.5     |
|    ep_rew_mean        | 86.5     |
| time/                 |          |
|    fps                | 1572     |
|    iterations         | 5500     |
|    time_elapsed       | 17       |
|    total_timesteps    | 27500    |
| train/                |          |
|

...

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.34e+03 |
|    ep_rew_mean        | 2.34e+03 |
| time/                 |          |
|    fps                | 1477     |
|    iterations         | 74500    |
|    time_elapsed       | 252      |
|    total_timesteps    | 372500   |
| train/                |          |
|    entropy_loss       | -0.0142  |
|    explained_variance | -0.0407  |
|    learning_rate      | 0.0007   |
|    n_updates          | 74499    |
|    policy_loss        | 1.46e-06 |
|    value_loss         | 7.43e-07 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.34e+03 |
|    ep_rew_mean        | 2.34e+03 |
| time/                 |          |
|    fps                | 1477     |
|    iterations         | 74600    |
|    time_elapsed       | 252      |
|    total_timesteps    | 373000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.34e+03 |
|    ep_rew_mean        | 2.34e+03 |
| time/                 |          |
|    fps                | 1478     |
|    iterations         | 75900    |
|    time_elapsed       | 256      |
|    total_timesteps    | 379500   |
| train/                |          |
|    entropy_loss       | -0.193   |
|    explained_variance | 0.0513   |
|    learning_rate      | 0.0007   |
|    n_updates          | 75899    |
|    policy_loss        | 6.28e-05 |
|    value_loss         | 5.8e-07  |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.34e+03 |
|    ep_rew_mean        | 2.34e+03 |
| time/                 |          |
|    fps                | 1478     |
|    iterations         | 76000    |
|    time_elapsed       | 256      |
|    total_timesteps    | 380000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.34e+03 |
|    ep_rew_mean        | 2.34e+03 |
| time/                 |          |
|    fps                | 1478     |
|    iterations         | 77300    |
|    time_elapsed       | 261      |
|    total_timesteps    | 386500   |
| train/                |          |
|    entropy_loss       | -0.206   |
|    explained_variance | 0.067    |
|    learning_rate      | 0.0007   |
|    n_updates          | 77299    |
|    policy_loss        | 0.000139 |
|    value_loss         | 5.99e-07 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.34e+03 |
|    ep_rew_mean        | 2.34e+03 |
| time/                 |          |
|    fps                | 1478     |
|    iterations         | 77400    |
|    time_elapsed       | 261      |
|    total_timesteps    | 387000   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.34e+03 |
|    ep_rew_mean        | 2.34e+03 |
| time/                 |          |
|    fps                | 1476     |
|    iterations         | 78600    |
|    time_elapsed       | 266      |
|    total_timesteps    | 393000   |
| train/                |          |
|    entropy_loss       | -0.17    |
|    explained_variance | -0.0315  |
|    learning_rate      | 0.0007   |
|    n_updates          | 78599    |
|    policy_loss        | 0.000131 |
|    value_loss         | 5.77e-07 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.34e+03 |
|    ep_rew_mean        | 2.34e+03 |
| time/                 |          |
|    fps                | 1477     |
|    iterations         | 78700    |
|    time_elapsed       | 266      |
|    total_timesteps    | 393500   |
| train/                |          |
|

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 2.34e+03 |
|    ep_rew_mean        | 2.34e+03 |
| time/                 |          |
|    fps                | 1478     |
|    iterations         | 80000    |
|    time_elapsed       | 270      |
|    total_timesteps    | 400000   |
| train/                |          |
|    entropy_loss       | -0.166   |
|    explained_variance | -0.126   |
|    learning_rate      | 0.0007   |
|    n_updates          | 79999    |
|    policy_loss        | 8.59e-05 |
|    value_loss         | 1.48e-06 |
------------------------------------


## Testing the model with rendering

In [34]:
env_rendering = CartPoleEnv(render_mode = "human")

In [35]:
obs = env_rendering.reset()
for i in range(1000):
    action, _state = model.predict(obs, deterministic=False)
    obs, reward, done, info = env_rendering.step(action)
    env_rendering.render()
    if done:
      obs = env_rendering.reset()

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default


In [36]:
env.close()
env_rendering.close()